## Build history of commodity COTs

1. Retrieve data from the CFTC website (www.cft.gov/files);
2. Extract data from the most important Commercial and Non Commercial long and short columns;
3. Create "net" columns for each important category;
4. Merge this data with the ETF history data created in the previous steps


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os, sys,glob
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import zipfile
import urllib.request
from PIL import Image
def str_to_date(d):
    try:
        dt = datetime.datetime.strptime(str(d),'%Y-%m-%d')
    except:
        return None
    return dt

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/cot')
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/zip')
except:
    pass


___
## First, decide if you want to re-create the ETF and COT data, or just retrieve the previously saved data DataFrames.

In [3]:
CREATE_COT_DATA = True
cot_save_path = './cot_new_history.csv'
cot_net_save_path = './cot_net_new_history.csv'
YEAR_OFFSET = 0 if datetime.datetime.now() > datetime.datetime(2020,1,10) else 1

In [4]:
YEAR_OFFSET

1

### Main column names

In [5]:
basic_cols = ['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD','Open_Interest_All']
long_cols = basic_cols + ['Prod_Merc_Positions_Long_All','Swap_Positions_Long_All',
            'M_Money_Positions_Long_All','Other_Rept_Positions_Long_All',
            'NonRept_Positions_Long_All','Tot_Rept_Positions_Long_All']
short_cols = basic_cols + ['Prod_Merc_Positions_Short_All','Swap_Positions_Short_All',
            'M_Money_Positions_Short_All','Other_Rept_Positions_Short_All',
            'NonRept_Positions_Short_All','Tot_Rept_Positions_Short_All']
summary_types = ['prod','swap','monman','other','nonrep','totrep']
summary_cols_dict = {summary_types[i]:(long_cols[i+len(basic_cols)],short_cols[i+len(basic_cols)]) for i in range(len(summary_types))}


In [6]:
summary_cols_dict

{'prod': ('Prod_Merc_Positions_Long_All', 'Prod_Merc_Positions_Short_All'),
 'swap': ('Swap_Positions_Long_All', 'Swap_Positions_Short_All'),
 'monman': ('M_Money_Positions_Long_All', 'M_Money_Positions_Short_All'),
 'other': ('Other_Rept_Positions_Long_All', 'Other_Rept_Positions_Short_All'),
 'nonrep': ('NonRept_Positions_Long_All', 'NonRept_Positions_Short_All'),
 'totrep': ('Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All')}

___
## Process CFTC COT Data
___

### Initial processing
1. Download zip files from www.cft.gov/files;
2. Unip the files using the zipfile package;
3. Read each csv (usually named Annual.TXT), and merge them into the df_cot DataFrame.


In [7]:
zip_download_folder = f'{TEMP_FOLDER}/zip'

In [8]:
f'{zip_download_folder}/*.csv'

'./temp_folder/zip/*.csv'

In [9]:
glob.glob(f'{zip_download_folder}/*')

['./temp_folder/zip/Annual.TXT',
 './temp_folder/zip/annual_2007.txt',
 './temp_folder/zip/dea_fut_xls_2000.0.zip',
 './temp_folder/zip/dea_fut_xls_2001.0.zip',
 './temp_folder/zip/dea_fut_xls_2002.0.zip',
 './temp_folder/zip/dea_fut_xls_2003.0.zip',
 './temp_folder/zip/dea_fut_xls_2004.0.zip',
 './temp_folder/zip/dea_fut_xls_2005.0.zip',
 './temp_folder/zip/dea_fut_xls_2006.0.zip',
 './temp_folder/zip/dea_fut_xls_2007.0.zip',
 './temp_folder/zip/dea_fut_xls_2008.0.zip',
 './temp_folder/zip/dea_fut_xls_2009.0.zip',
 './temp_folder/zip/dea_fut_xls_2010.0.zip',
 './temp_folder/zip/dea_fut_xls_2011.0.zip',
 './temp_folder/zip/dea_fut_xls_2012.0.zip',
 './temp_folder/zip/dea_fut_xls_2013.0.zip',
 './temp_folder/zip/dea_fut_xls_2014.0.zip',
 './temp_folder/zip/dea_fut_xls_2015.0.zip',
 './temp_folder/zip/dea_fut_xls_2016.0.zip',
 './temp_folder/zip/dea_fut_xls_2017.0.zip',
 './temp_folder/zip/dea_fut_xls_2018.0.zip',
 './temp_folder/zip/dea_fut_xls_2019.0.zip',
 './temp_folder/zip/f_year.tx

In [10]:
if CREATE_COT_DATA:
    last_year = datetime.datetime.now().year
    year_list = np.arange(2010,last_year+1)
    zip_download_folder = f'{TEMP_FOLDER}/zip'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
#         url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip" 
        url = f'https://www.cftc.gov/files/dea/history/fut_disagg_txt_{yint}.zip'
        path_to_zip_file = f'{zip_download_folder}/fut_disagg_txt_{y}.zip'
        if len(glob.glob(f'{zip_download_folder}/fut_disagg_txt_{y}.csv'))>0:
            print(f'Already downloaded fut_disagg_txt_{y}.csv')
            continue
        print(f'About to downloaded fut_disagg_txt_{y}.csv')


About to downloaded fut_disagg_txt_2010.csv
About to downloaded fut_disagg_txt_2011.csv
About to downloaded fut_disagg_txt_2012.csv
About to downloaded fut_disagg_txt_2013.csv
About to downloaded fut_disagg_txt_2014.csv
About to downloaded fut_disagg_txt_2015.csv
About to downloaded fut_disagg_txt_2016.csv
About to downloaded fut_disagg_txt_2017.csv
About to downloaded fut_disagg_txt_2018.csv
About to downloaded fut_disagg_txt_2019.csv
About to downloaded fut_disagg_txt_2020.csv


In [11]:
if CREATE_COT_DATA:
    last_year = datetime.datetime.now().year - YEAR_OFFSET
    year_list = np.arange(2010,last_year+1)
    zip_download_folder = f'{TEMP_FOLDER}/zip'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
#         url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip" 
        url = f'https://www.cftc.gov/files/dea/history/fut_disagg_txt_{yint}.zip'
        path_to_zip_file = f'{zip_download_folder}/fut_disagg_txt_{y}.zip'
        if len(glob.glob(f'{zip_download_folder}/fut_disagg_txt_{y}.csv'))>0:
            print(f'Already downloaded fut_disagg_txt_{y}.csv')
            continue
        if not os.path.isfile(path_to_zip_file) or y >= last_year:
            print(f'retrieving cot zip file from {url}')
            try:
                urllib.request.urlretrieve(url, path_to_zip_file)    
            except:
                import time
                time.sleep(1)
                urllib.request.urlretrieve(url, path_to_zip_file)    
        zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
        zip_ref.extractall(zip_download_folder)
        zip_ref.close()
#         df_cot_temp = pd.read_csv(f'{zip_download_folder}/Annual.TXT')
        df_cot_temp = pd.read_csv(f'{zip_download_folder}/f_year.txt')
        if df_cot is None:
            df_cot = df_cot_temp.copy()
        else:
            df_cot = df_cot.append(df_cot_temp,ignore_index=True)
            df_cot.index = list(range(len(df_cot)))
        print(f'processed cot csv file from {url}. Length = {len(df_cot_temp)}')
        
df_cot.head()

processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2010.zip. Length = 5547
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2011.zip. Length = 5486
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2012.zip. Length = 5938
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2013.zip. Length = 8270
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2014.zip. Length = 10271
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2015.zip. Length = 9762
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2016.zip. Length = 10520
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2017.zip. Length = 10204
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2018.zip. Length = 10874
retrieving cot zip file from https://www.c

,As_of_Date_In_Form_YYMMDD,CFTC_Commodity_Code,CFTC_Commodity_Code_Quotes,CFTC_Contract_Market_Code,CFTC_Contract_Market_Code_Quotes,CFTC_Market_Code,CFTC_Market_Code_Quotes,CFTC_Region_Code,CFTC_SubGroup_Code,Change_in_M_Money_Long_All,...,Traders_Swap_Spread_Other,Traders_Tot_All,Traders_Tot_Old,Traders_Tot_Other,Traders_Tot_Rept_Long_All,Traders_Tot_Rept_Long_Old,Traders_Tot_Rept_Long_Other,Traders_Tot_Rept_Short_All,Traders_Tot_Rept_Short_Old,Traders_Tot_Rept_Short_Other
0,101228,1,1,001602,001602,CBT,CBT,0,A10,4819,...,21,413,393,272,310,251,162,297,243,217
1,101221,1,1,001602,001602,CBT,CBT,0,A10,-1932,...,21,406,383,274,302,238,160,299,244,217
2,101214,1,1,001602,001602,CBT,CBT,0,A10,3201,...,21,421,396,281,307,244,159,312,256,221
3,101207,1,1,001602,001602,CBT,CBT,0,A10,18531,...,21,415,397,274,308,242,157,298,256,215
4,101130,1,1,001602,001602,CBT,CBT,0,A10,-17,...,19,408,387,266,284,231,140,329,282,215


___
### Make column names easier to process, make main date field a datetime object, and sort the DataFrame
___

___
### Show important columns for a specific  commodity
___

In [12]:
if CREATE_COT_DATA:
    col_rename_dict = {c:c.strip().replace('__','_').replace(' ','_').replace('-','_').replace('(','').replace(')','') for c in df_cot.columns.values}
    df_cot2 = df_cot.rename(columns=col_rename_dict)
    df_cot2 = df_cot2.drop(columns=['Report_Date_as_MM_DD_YYYY'])
    df_cot2.Market_and_Exchange_Names = df_cot2.Market_and_Exchange_Names.str.strip()
    l = lambda s:datetime.datetime(2000+int(str(s)[0:2]),int(str(s)[2:4]),int(str(s)[4:6]))
    df_cot2.As_of_Date_In_Form_YYMMDD = df_cot2.As_of_Date_In_Form_YYMMDD.apply(l)
    df_cot2 = df_cot2.sort_values(['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD'])
    df_cot2.to_csv(cot_save_path,index=False)
    
    

In [13]:
df_cot2 = pd.read_csv(cot_save_path)
df_cot2.As_of_Date_In_Form_YYMMDD = df_cot2.As_of_Date_In_Form_YYMMDD.apply(str_to_date)
cot_beg_date = datetime.datetime.now() - datetime.timedelta(2000)
# df_commod = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains(commod)][df_cot2.As_of_Date_In_Form_YYMMDD>=cot_beg_date]
df_commod = df_cot2[df_cot2.As_of_Date_In_Form_YYMMDD>=cot_beg_date]
df_commod_basic = df_commod[basic_cols]
df_commod_long = df_commod[long_cols]
df_commod_short = df_commod[short_cols]


### Show basic open interest info

In [14]:
commod = 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE'
[c for c in df_commod.Market_and_Exchange_Names.values if 'CRUDE' in c]

['BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK ME

In [15]:
df_commod_basic[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()


,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All
14354,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-24,2114661
14353,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-17,2166584
14352,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-10,2229813
14351,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-03,2163757
14350,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-11-26,2187168


### Show important "long" position info

In [16]:
df_commod_long[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
14354,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-24,2114661,399370,135930,324992,300310,83531,2031130
14353,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-17,2166584,439892,133364,310460,301653,87948,2078636
14352,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-10,2229813,493855,127877,271959,305257,88253,2141560
14351,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-03,2163757,472745,130867,214170,316950,83413,2080344
14350,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-11-26,2187168,468648,123924,244968,308769,82124,2105044


### Show important "short" position info

In [17]:
df_commod_short[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Short_All,Swap_Positions_Short_All,M_Money_Positions_Short_All,Other_Rept_Positions_Short_All,NonRept_Positions_Short_All,Tot_Rept_Positions_Short_All
14354,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-24,2114661,417347,676822,31272,43953,74739,2039922
14353,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-17,2166584,465687,649532,25131,50605,82362,2084222
14352,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-10,2229813,513544,610177,30942,50735,81803,2148010
14351,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-03,2163757,483558,557764,62981,40104,73738,2090019
14350,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-11-26,2187168,486425,580923,35781,47020,78284,2108884


### CREATE important "net" position info for other anlysis notebooks

In [18]:
df_commod_long.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
35,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-15,304676,90264,55279,34165,21840,36602,268074
36,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-22,329082,108775,58396,30830,19191,37123,291959
37,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-29,336160,107150,58715,30368,19924,38826,297334
38,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-05,342910,108110,59518,27308,21053,38115,304795
39,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-12,356605,98600,61870,31164,22835,41748,314857


In [19]:
df_commod_long.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
35,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-15,304676,90264,55279,34165,21840,36602,268074
36,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-22,329082,108775,58396,30830,19191,37123,291959
37,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-29,336160,107150,58715,30368,19924,38826,297334
38,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-05,342910,108110,59518,27308,21053,38115,304795
39,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-12,356605,98600,61870,31164,22835,41748,314857


In [20]:
df_commod_short.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Short_All,Swap_Positions_Short_All,M_Money_Positions_Short_All,Other_Rept_Positions_Short_All,NonRept_Positions_Short_All,Tot_Rept_Positions_Short_All
35,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-15,304676,144043,2853,24306,36331,30617,274059
36,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-22,329082,155312,3008,31021,30107,34867,294215
37,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-07-29,336160,160048,4839,37686,21250,31160,305000
38,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-05,342910,159492,4281,44239,14325,31767,311143
39,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2014-08-12,356605,161693,4879,42500,14415,32730,323875


In [21]:
df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=basic_cols)
print(len(df_commod_net),len(df_commod_long),len(df_commod_short))


57033 57033 57033


In [22]:
df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=basic_cols)
print(len(df_commod_net),len(df_commod_long),len(df_commod_short))

net_cols = [c1 for c1 in long_cols + short_cols if c1 not in basic_cols]
for c2 in net_cols:
    df_commod_net = df_commod_net[df_commod_net[c2].astype(str).str.contains('[0-9]')]
print(len(df_commod_net))


for p in summary_cols_dict.keys():
    t = summary_cols_dict[p]
    lc = t[0]
    sc = t[1]
    df_commod_net[p+'_net'] = df_commod_net[lc].astype(float) - df_commod_net[sc].astype(float)
    df_commod_net[p+'_ratio'] = df_commod_net[lc].astype(float) / df_commod_net[sc].astype(float)
print(len(df_commod_net))

sort_cols = ['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD']
df_commod_net = df_commod_net.sort_values(sort_cols)


57033 57033 57033
57033
57033


In [23]:
ratio_cols = ['prod_ratio','monman_ratio','swap_ratio','other_ratio','nonrep_ratio']
df_commod_net[df_commod_net.Market_and_Exchange_Names.str.contains(commod)][sort_cols + ratio_cols].sort_values(sort_cols,ascending=False).head()



,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,prod_ratio,monman_ratio,swap_ratio,other_ratio,nonrep_ratio
9259,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-24,0.956926,10.392428,0.200836,6.832526,1.117636
9258,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-17,0.944609,12.353667,0.205323,5.960933,1.067823
9257,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-10,0.961661,8.789315,0.209574,6.016695,1.078848
9256,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-12-03,0.977639,3.400549,0.234628,7.903202,1.131208
9255,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2019-11-26,0.963454,6.846315,0.213323,6.566759,1.049052


In [24]:
nan_cols = df_commod_net.columns[df_commod_net.isna().any()].tolist()
if len(nan_cols)>0:
    print(nan_cols)
    df_commod_net = df_commod_net.fillna(0)
    print(df_commod_net[df_commod_net.monman_ratio.isnull()][ratio_cols])

['swap_ratio', 'monman_ratio', 'other_ratio', 'nonrep_ratio']
Empty DataFrame
Columns: [prod_ratio, monman_ratio, swap_ratio, other_ratio, nonrep_ratio]
Index: []


In [25]:
df_commod_net.columns.values

array(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
       'Open_Interest_All', 'Prod_Merc_Positions_Long_All',
       'Swap_Positions_Long_All', 'M_Money_Positions_Long_All',
       'Other_Rept_Positions_Long_All', 'NonRept_Positions_Long_All',
       'Tot_Rept_Positions_Long_All', 'Prod_Merc_Positions_Short_All',
       'Swap_Positions_Short_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Short_All', 'NonRept_Positions_Short_All',
       'Tot_Rept_Positions_Short_All', 'prod_net', 'prod_ratio',
       'swap_net', 'swap_ratio', 'monman_net', 'monman_ratio',
       'other_net', 'other_ratio', 'nonrep_net', 'nonrep_ratio',
       'totrep_net', 'totrep_ratio'], dtype=object)

In [26]:
print(f'saving {len(df_commod_net)} records')
df_commod_net.to_csv(cot_net_save_path,index=False)

saving 57033 records


## END

In [27]:
len(df_commod_net[df_commod_net.Market_and_Exchange_Names.str.contains('SILVER - COMMODITY EXCHANGE INC.')])

285

In [28]:
df_cot2[df_cot2.Market_and_Exchange_Names.str.lower().str.contains('10')].Market_and_Exchange_Names.unique()


array(['1000 CALIFORNIA CARBON ALLOWANCE - ICE FUTURES ENERGY DIV',
       'HENRY HUB NAT GAS FINL-10000 - NASDAQ FUTURES',
       'HHUB NAT GAS PENULT FINL-10000 - NASDAQ FUTURES',
       'NFX SM10TC SUPRAMAX 10T/C AVG - NASDAQ FUTURES'], dtype=object)

In [29]:
list(filter(lambda s:'As_of_Date_In_Form_YYMMDD' in s,df_cot2.columns.values))

['As_of_Date_In_Form_YYMMDD']

In [30]:
df_wheat = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains('WHEAT-SRW - CHICAGO BOARD OF TRADE')]
len(df_wheat)
# np.sort(df_wheat.As_of_Date_In_Form_YYMMDD.unique())

315

In [31]:
df_cot2[df_cot2.Market_and_Exchange_Names.str.contains('WHEAT-HRW - CHICAGO BOARD OF TRADE')].As_of_Date_In_Form_YYMMDD.max()


Timestamp('2019-12-24 00:00:00')